In [114]:
import pandas as pd
import random
VERSION_NUMBER = 1

In [115]:
def roll_dice(type_dice):
    total = 0
    total += random.randint(1,type_dice)
    return total

In [186]:
class Titan:
    def __init__(self, name, ac, hp, extermination_attack):
        self.name = name
        self.ac = ac
        self.total_hp = hp
        self.hp = hp
        self.extermination_attack = extermination_attack
        self.dead = dead
        
    def exterminate(self, people):
        victims = roll_dice(self.extermination_attack)
        people.distribute_victims(victims)
        
    def lose_hp(self, wound):
        self.hp -= wound
        if self.hp<=0:
            self.die()
    
    def die(self):
        self.dead = True
        
        
class People:
    def __init__ (self, soldiers, ackermans):
        self.total_soldiers = soldiers
        self.soldiers = soldiers
        self.total_ackermans = ackermans
        self.ackermans = ackermans
        self.exterminated = False
    
    def distribute_victims(self, victims):
        if self.soldiers > 0 and self.soldiers >= victims:
            self.soldiers -= victims
        elif self.soldiers > 0 and self.soldiers < victims:
            self.ackermans -= victims - self.soldiers
            self.soldiers = 0
        elif  self.soldiers <= 0:
            self.ackermans  -= victims
        if self.soldiers <= 0 and self.ackermans <= 0:
            self.exterminated = True
            
    def attack(self, titan):
        if self.ackermans > 0:
            for i in range(self.ackermans):
                total_attack = roll_dice(20) + self.ackermans
                if total_attack > titan.ac:
                    total_damage = roll_dice(6)+ self.ackermans
                    titan.lose_hp(total_damage)

In [210]:
class AttackTitan(Titan):
    def __init__(self):
        self.name = 'Attack Titan'
        self.ac = 15
        self.total_hp = 50
        self.hp = 50
        self.dead = False
        self.extermination_attack = 3
        
        
class ArmouredTitan(Titan):
    def __init__(self):
        self.name = 'Armoured Titan'
        self.ac = 17
        self.total_hp = 60
        self.hp = 60
        self.dead = False
        self.shield = 5
        self.extermination_attack = 6
        
    def lose_hp(self, wound):
        if self.shield <= 0:
            self.hp -= wound
        else:
            wound -= self.shield
            self.shield -= wound
            if wound > 0:
                self.hp -= wound
        if self.hp<=0:
            self.die()
 

class ColossalTitan(Titan):
    def __init__(self):
        self.name = 'Colossal Titan'
        self.ac = 12
        self.total_hp = 100
        self.hp = 100
        self.dead = False
        self.extermination_attack = 3
        
        
class BeastTitan(Titan):
    def __init__(self):
        self.name = 'Beast Titan'
        self.ac = 13
        self.total_hp = 40
        self.hp = 40
        self.dead = False
        self.extermination_attack = 6      
       
    
class WarhammerTitan(Titan):
    def __init__(self):
        self.name = 'Warhammer Titan'
        self.ac = 15
        self.total_hp = 50
        self.hp = 50
        self.dead = False
        self.extermination_attack = 3
        
    def lose_hp(self, wound):
        self.hp -= wound
        if self.hp<=0:
            if roll_dice(100, 1) >= 90:
                self.hp = 10
            else:
                self.die()
                
                
class FemaleTitan(Titan):
    def __init__(self):
        self.name = 'Female Titan'
        self.ac = 15
        self.total_hp = 45
        self.hp = 45
        self.dead = False
        self.extermination_attack = 3
        
        
class CartTitan(Titan):
    def __init__(self):
        self.name = 'Cart Titan'
        self.ac = 13
        self.total_hp = 40
        self.hp = 40
        self.dead = False
        self.extermination_attack = 6

class JawTitan(Titan):
    def __init__(self):
        self.name = 'Jaw Titan'
        self.ac = 15
        self.total_hp = 45
        self.hp = 45
        self.dead = False
        self.extermination_attack = 3
        
    def lose_hp(self, wound):
        if roll_dice(100, 1) < 80:
            self.hp -= wound
            if self.hp<=0:
                self.die()

In [211]:
def combat_phase(titan, people):
    if not people.exterminated:
        people.attack(titan)
    if not titan.dead:
        titan.exterminate(people)

def duel(titan, people):
    stats = {}
    turns = 0
    while not titan.dead and not people.exterminated:
        combat_phase(titan, people)
        turns += 1
    stats = calculate_stats(titan, people, turns)
    return stats
    
def calculate_stats(titan, people, turns):
    stats = {}
    if titan.hp < 0:
        titan.hp = 0
    if people.soldiers < 0:
        people.soldiers = 0
    if people.ackermans < 0:
        people.ackermans = 0
    if titan.dead:
        stats['winner'] = 'people'
    elif people.exterminated:
        stats['winner'] = 'titan'
    else:
        stats['winner'] = 'NONE'
    stats['turns'] = turns
    stats['titan_total_damage'] = titan.total_hp - titan.hp
    stats['titan_mean_damage'] = round(stats['titan_total_damage']/turns, 2)
    stats['dead_soldiers'] = people.total_soldiers - people.soldiers
    stats['dead_ackermans'] = people.total_ackermans - people.ackermans
    return stats
        

In [212]:
people = People(0,6)
titan = AttackTitan()
duel(titan,people)

{'winner': 'people',
 'turns': 2,
 'titan_total_damage': 50,
 'titan_mean_damage': 25.0,
 'dead_soldiers': 0,
 'dead_ackermans': 1}

In [213]:
def calculate_stats_from_sample(sample_stats_rows,soldiers,ackerman_units):
    titan_win_rate = 0
    titan_wins = 0
    people_win_rate = 0
    people_wins = 0
    total_turns = 0
    avg_turns = 0
    total_titan_hp_lost = 0
    avg_titan_hp_lost = 0
    for row in sample_stats_rows:
        if row['winner'] == 'titan':
            titan_wins += 1
        else:
            people_wins += 1
        total_turns += row['turns']
        total_titan_hp_lost += row['titan_total_damage']
    titan_win_rate = titan_wins/len(sample_stats_rows)
    people_win_rate = people_wins/len(sample_stats_rows)
    avg_turns = total_turns/len(sample_stats_rows)
    avg_titan_hp_lost = total_titan_hp_lost/len(sample_stats_rows)
    stats = {
        'soldiers':soldiers,
        'ackerman':ackerman_units,
        'titan_win_rate':titan_win_rate,
        'people_win_rate':people_win_rate,
        'avg_turns':avg_turns,
        'avg_titan_hp_lost':avg_titan_hp_lost
    }
    return stats

In [214]:
population_size = [3,5,8,10]
all_stats_rows = []
sample_stats_rows = []
sample_size = 100
for population in population_size:
    ackerman_units = 1
    for i in range(population):
        for i in range(sample_size):
            titan = AttackTitan()
            people = People(population-ackerman_units, ackerman_units)
            sample_stats_row = duel(titan,people)
            sample_stats_rows.append(sample_stats_row)
        stats_row = calculate_stats_from_sample(sample_stats_rows,population-ackerman_units,ackerman_units)
        all_stats_rows.append(stats_row)
        ackerman_units += 1
        
df = pd.DataFrame(data=all_stats_rows)

In [215]:
heatmap = df.style.background_gradient(axis=0)
heatmap

,soldiers,ackerman,titan_win_rate,people_win_rate,avg_turns,avg_titan_hp_lost
0,2,1,1.000000,0.000000,1.770000,2.290000
1,1,2,1.000000,0.000000,1.680000,3.945000
2,0,3,1.000000,0.000000,1.733333,5.873333
3,4,1,1.000000,0.000000,2.005000,5.357500
4,3,2,1.000000,0.000000,2.182000,6.210000
5,2,3,1.000000,0.000000,2.305000,7.906667
6,1,4,0.992857,0.007143,2.381429,10.361429
7,0,5,0.981250,0.018750,2.436250,13.040000
8,7,1,0.983333,0.016667,2.655556,12.374444
9,6,2,0.985000,0.015000,2.818000,12.620000


In [203]:
# Salva la heatmap in un file html
html_str = heatmap.render()
Html_file= open('output_html\\titan_vs_people\\'+'attack_vs_people_v'+str(VERSION_NUMBER)+'.html','w')
Html_file.write(html_str)
Html_file.close()